In [3]:
import pylogit as pl
import pandas as pd
import numpy as np
import json

In [8]:
state_codes={'Detroit': 'mi', 'Boston': 'ma'}
state_fips={'Detroit': '26', 'Boston': '25'}
NUM_ALTS=8
sample_size=5000

PUMA_POP_PATH='./cities/'+'Detroit'+'/raw/ACS/population.csv'
# https://www2.census.gov/programs-surveys/acs/data/pums/2016/1-Year/
PUMS_HH_PATH='./cities/'+'Detroit'+'/raw/PUMS/csv_h'+state_codes['Detroit']+'/ss16h'+state_codes['Detroit']+'.csv'
PUMS_POP_PATH='./cities/'+'Detroit'+'/raw/PUMS/csv_p'+state_codes['Detroit']+'/ss16p'+state_codes['Detroit']+'.csv'
#        POI_PATH = './cities/'+self.city_folder+'/raw/OSM/poi.geojson'
PUMA_TO_POW_PUMA_PATH='./puma_to_pow_puma.csv'
PUMA_SHAPE_PATH = './cities/'+'Detroit'+'/raw/pumas.geojson'

In [20]:
puma_land_sqm={str(int(f['properties']['PUMACE10'])).zfill(5): f['properties']['ALAND10']
                for f in pumas_shape['features']}

In [21]:
puma_land_sqm

{'00100': 22265283522,
 '03201': 276074457,
 '02702': 76441098,
 '02102': 146739375,
 '01003': 746189811,
 '01100': 6211311259,
 '01704': 2107917012,
 '03006': 93561031,
 '03208': 80928884,
 '02800': 1463690104,
 '01801': 1346622162,
 '01900': 2956695235,
 '01004': 1088614443,
 '01200': 4417158073,
 '01702': 280587095,
 '02902': 542533049,
 '03003': 94553340,
 '01002': 113978646,
 '01703': 86531426,
 '02903': 177916109,
 '03209': 52358485,
 '03204': 99887413,
 '02901': 712027237,
 '02905': 181566099,
 '03202': 121558970,
 '02908': 98764724,
 '02907': 165020377,
 '02904': 179555507,
 '02906': 189325805,
 '01802': 93724700,
 '00500': 4334083319,
 '01400': 2482657514,
 '03001': 664854579,
 '03005': 69408445,
 '00400': 7897322260,
 '02701': 1462964684,
 '02000': 3261757857,
 '00802': 846614851,
 '03002': 186994314,
 '03203': 93219391,
 '03205': 341660197,
 '02703': 289022601,
 '02600': 1817307823,
 '02101': 1309342343,
 '01001': 246670361,
 '01500': 2073181589,
 '01701': 2217854406,
 '0321

In [70]:
hh=pd.read_csv(PUMS_HH_PATH)
pop = pd.read_csv(PUMS_POP_PATH)
hh['PUMA']=hh.apply(lambda row: str(int(row['PUMA'])).zfill(5), axis=1)
pop['PUMA']=pop.apply(lambda row: str(int(row['PUMA'])).zfill(5), axis=1)
pop['POWPUMA']=pop.apply(lambda row: str(int(row['POWPUMA'])).zfill(5) 
                        if not np.isnan(row['POWPUMA']) else 'NaN', axis=1)

#        all_PUMAs=list(set(hh['PUMA']))
# pumas_included=json.load(open(self.PUMAS_INCLUDED_PATH))                                         # For the whole MI
pumas_shape=json.load(open(PUMA_SHAPE_PATH))
pumas_order=[f['properties']['PUMACE10'] for f in pumas_shape['features']]
            
puma_pop=pd.read_csv(PUMA_POP_PATH)
puma_pop['PUMA']=puma_pop.apply(lambda row: str(row['PUMA']).zfill(5), axis=1)
puma_pop=puma_pop.set_index('PUMA')


# identify recent movers and vacant houses                                            
# hh_vacant_for_rent=hh[(hh['VACS']==1) & (hh['PUMA'].isin(pumas_included))].copy()          
# hh_rented=hh[(hh['TEN']==3) & (hh['PUMA'].isin(pumas_included))].copy()                                                      
# renters_recent_move=hh_rented[hh_rented['MV']==1].copy()     

# get the area of each PUMA
puma_land_sqm=dict(zip(puma_pop.index, puma_pop['AREALAND'].astype('int64')))


In [37]:
{str(int(f['properties']['PUMACE10'])).zfill(5): f['properties']['ALAND10']
                        for f in pumas_shape['features']}

{'00100': 22265283522,
 '03201': 276074457,
 '02702': 76441098,
 '02102': 146739375,
 '01003': 746189811,
 '01100': 6211311259,
 '01704': 2107917012,
 '03006': 93561031,
 '03208': 80928884,
 '02800': 1463690104,
 '01801': 1346622162,
 '01900': 2956695235,
 '01004': 1088614443,
 '01200': 4417158073,
 '01702': 280587095,
 '02902': 542533049,
 '03003': 94553340,
 '01002': 113978646,
 '01703': 86531426,
 '02903': 177916109,
 '03209': 52358485,
 '03204': 99887413,
 '02901': 712027237,
 '02905': 181566099,
 '03202': 121558970,
 '02908': 98764724,
 '02907': 165020377,
 '02904': 179555507,
 '02906': 189325805,
 '01802': 93724700,
 '00500': 4334083319,
 '01400': 2482657514,
 '03001': 664854579,
 '03005': 69408445,
 '00400': 7897322260,
 '02701': 1462964684,
 '02000': 3261757857,
 '00802': 846614851,
 '03002': 186994314,
 '03203': 93219391,
 '03205': 341660197,
 '02703': 289022601,
 '02600': 1817307823,
 '02101': 1309342343,
 '01001': 246670361,
 '01500': 2073181589,
 '01701': 2217854406,
 '0321

In [62]:
puma_land_sqm

{'00200': 5251107983,
 '00300': 4752896547,
 '00401': 134042365,
 '00402': 1051593877,
 '00403': 163234632,
 '00501': 215207507,
 '00600': 11759818850,
 '00700': 1180428103,
 '00800': 5924555475,
 '00900': 4230400126,
 '01000': 12124727452,
 '01100': 1587662015,
 '01201': 3327822863,
 '01202': 171152347,
 '01301': 2222480621,
 '01401': 90404025,
 '01402': 141830034,
 '01403': 97830127,
 '01404': 107243591,
 '01501': 2042228282,
 '01502': 935744353,
 '01600': 7458715516,
 '01700': 1045344987,
 '01801': 1519766008,
 '01802': 314723915,
 '01901': 166693887,
 '02000': 4495506197,
 '02101': 182480134,
 '02201': 307705579,
 '02300': 4583601107,
 '02400': 503907543,
 '02500': 829870735,
 '02600': 1857268910,
 '02701': 2777531326,
 '02702': 1340124873,
 '02801': 2630013618,
 '02802': 327044185,
 '02803': 227006336,
 '00101': 100121433,
 '00102': 198186907,
 '00400': 392739381,
 '00103': 206269595,
 '00104': 787124785,
 '00105': 8518845,
 '00106': 28437903,
 '00107': 105397842,
 '00108': 708959